# PRÁCTICA 2 DE MH
## Cristina Sánchez Justicia

Librerias que van a ser usadas: 
- numpy: Para trabajar con matrices
- arff: Lectura de archivos
- tabulate: salida 

In [12]:
import numpy as np 
from scipy.io import arff
from tabulate import tabulate

# Operador de cruce blx
Parámetros:
- madre, padre: dos vectores de pesos 

Return: 
- numpy array con dos vectores de pesos descendientes de esos padres

In [13]:
def blx(madre, padre): 
  tam = len(madre)
  ALPHA = 0.3
  # Vectores con Cmin y Cmax correspondientes para cada peso 
  min = np.empty(tam) 
  max = np.empty(tam)
  for i in range(tam):
    minimo = np.minimum(madre[i], padre[i])
    maximo = np.maximum(madre[i], padre[i])
    I = maximo - minimo 
    min[i] = minimo - I*ALPHA
    max[i] = maximo + I*ALPHA
  devolver1 = np.empty(tam)
  devolver2 = np.empty(tam)  
  for i in range(tam):
    devolver1[i] = np.random.uniform(min[i],max[i]) 
    devolver2[i] = np.random.uniform(min[i],max[i]) 
  return devolver1, devolver2 

# Operador de cruce CA 
Parámetros: 
- madre, padre: dos vectores de pesos 

Return: 
- numpy array con dos vectores de pesos descendientes de los padres

In [14]:
def ca(madre, padre): 
  tam = len(madre)
  devolver = np.empty(tam)
  for i in range(len): 
    aleatorio = np.random.rand()
    devolver[i] = aleatorio * madre[i] + (1 - aleatorio)*padre[i]
  return devolver 


# 1-NN con Leave one out 
Código prestado por Pablo Millán Cubero

Parámetros: 
- datos: matriz de datos sin la columna de la etiqueta. Cada ejemplo es una fila. 
- clases: vector unidimensional de etiquetas que coincide con la etiqueta de cada fila
- w: peso que se quiere probar 

Return: 
- tas_clas

In [15]:
#Devuelve la predicción de clase del ejemplo 'nuevo', version alternativa
def classf_1NN_(nuevo, datos, w, i):
  distancias = np.sum((datos-nuevo)**2*w, 1)**0.5
  distancias[i] = np.Inf
  return np.argmin(distancias)

#Calcula el error sobre un conjunto (leave one out)
def calcular_error_leave(datos, clases, w):

  w_temp = w.copy()
  w_temp[w_temp < 0.1] = 0

  aciertos = 0
  for i in range(0, len(datos)):
    c = classf_1NN_(datos[i], datos, w_temp, i)
    if (clases[c] == clases[i]):
      aciertos += 1

  return aciertos/len(datos)

# Tasa de clasificación 
Mide la presición del clasificador
$$tasa\_clas = 100 \times \frac{nº correctas}{nº Totales}$$

In [16]:
def tasa_clas(pesos, datos, labels): 
  # devolver =  calcular_error_leave(datos, labels, pesos)
  # No me funciona lo de arriba 
  devolver = 0.5 
  return devolver 

# Tasa de reducción 
Mide la simplicidad del clasificador 
$$tasa\_red=100 \times \frac{nºvalores<0.1}{nºcaracterísticas}$$

In [17]:
def tasa_red(pesos): 
  UMBRAL = 0.1
  # (array < valor) convierte el array de pesos a un array de booleanos que son true sii cumplen la condición 
  # np.count_nonzero() devuelve el número de true que hay en un vector 
  return np.count_nonzero(pesos < UMBRAL) / len(pesos)

# Operador de evaluación 
Paráemtros: 
- 

In [18]:
def evaluacion(pesos, datos, labels): 
  ALPHA = 0.5
  return ALPHA * tasa_clas(pesos, datos, labels) + ALPHA * tasa_red(pesos)

# Mutación 
Parámetros: 
- pesos: array de pesos 

Salida: 
- array de pesos mutado 

In [19]:
def mutar(pesos):
  return pesos*np.random.uniform(0,1)

# Algoritmo genético generacional
Parámetros: 
- datos: ejemplos del problema  
- labels: la clase correspondiente de cada fila de características 
- opcruce: puntero a la función que hace de operador de cruce (blx o ca) 
- numero_iteraciones: cantidad de iteraciones que dará el algoritmo  

Return: 
- vector de pesos ajustado 

In [20]:
def agg (datos, labels, opcruce, numero_iteraciones_max):
  PROBABILIDAD_MUTACION = 0 
  # Se extrae el tamaño del vector de pesos de los datos del problema 
  tam_cromosoma = len(datos)
  # Número de cromosomas que hay en nuestra población 
  tam_poblacion = 10

  # inicializar P(t) 
  poblacion = np.random.rand(tam_poblacion, tam_cromosoma)
  
  iteraciones = 0
  while (iteraciones < numero_iteraciones_max): 
    poblacion_actual = np.empty((tam_poblacion,tam_cromosoma))
    for i in range(0, tam_poblacion, 2):  
      # recombinar P'
      hijo1, hijo2 = opcruce(poblacion[i], poblacion[i+1])
      # Solo se muta uno de ellos 
      if (i == 1): 
        poblacion_actual[i] = mutar(hijo1)      
      else: 
        poblacion_actual[i] = hijo1
      poblacion_actual[i+1] = hijo2
    evaluacion_poblacion = np.empty((tam_poblacion))
    evaluacion_actual = np.empty((tam_poblacion))
    # Elitismo. Se busca el mejor de P(t) y se cambia por el peor de P(t-1)
    for j in range(tam_poblacion): 
      evaluacion_poblacion[j] = evaluacion(poblacion[j], datos, labels)
      evaluacion_actual[j] = evaluacion(poblacion_actual[j], datos, labels)
      iteraciones += 1
    indice_maximo = np.argmax(evaluacion_poblacion)
    indice_minimo = np.argmin(evaluacion_actual)
    poblacion_actual[indice_minimo] = poblacion[indice_maximo]
    # reemplazar P(t) a partir de P(t-1) y P' 
    poblacion = poblacion_actual.copy()
  # Se buscan los mejores para devolver. Se ordena en orden descendiente.  
  poblacion[::-1].sort()
  return poblacion[0], poblacion[1]

# Algoritmo Genético Estacionario 
Parámetros: 
- datos: ejemplos del problema  
- labels: la clase correspondiente de cada fila de características 
- opcruce: puntero a la función que hace de operador de cruce (blx o ca) 
- numero_iteraciones: cantidad de iteraciones que dará el algoritmo  

Return: 
- vector de pesos ajustado 

In [21]:
def age (datos, labels, opcruce, numero_iteraciones): 
  resultado = np.empty(len(datos[0]))
  iteraciones = 0 
  poblacion = np.random.rand((50, len(datos[0])))
  while iteraciones < numero_iteraciones: 
    # Los dos números podrían ser el mismo 
    while True: 
      n = np.random.rand(0, len(datos[0]))
      m = np.random.rand(0, len(datos[0]))
      if n != m: 
        break 
    hijo1, hijo2 = opcruce(poblacion[n], poblacion[m])
    # Los hijos y los padres compiten y dejamos en la población los mejores
    competicion = np.empty(4)
    competicion[0] = evaluacion(poblacion[n], datos, labels)
    competicion[1] = evaluacion(poblacion[m], datos, labels)
    competicion[2] = evaluacion(hijo1, datos, labels)
    competicion[3] = evaluacion(hijo2, datos, labels)
    mejor1 = np.amax(competicion)
    competicion[np.where(competicion == mejor1)] = 0
    mejor2 = np.amax(competicion) 
    poblacion[n] = mejor1
    poblacion[m] = mejor2
    iteraciones += 1 
  return resultado 

# Programa main

In [22]:
# Lectura de datos 
print("¿Qué datos quiere leer?")
print("1- ionosphere")
print("2- parkinsons")
print("3- spectf-heart")
print("0- generacion de datos para tablas")
i = input("Introduzca 0, 1, 2, 3: ")
i = int(i)
while int(i) < 0 or int(i) > 3: 
  i = input("Pruebe otra vez: ")

ionosphere = "./datos/ionosphere.arff"
parkinsons = "./datos/parkinsons.arff"
heart = "./datos/spectf-heart.arff"

if i > 0 and i < 4:  
  tam_atributos = 0
  archivo = "" 
  if (int(i) == 1): 
    archivo = ionosphere 
    tam_atributos = 34
    cantidad_atributos = 351
  if (int(i) == 2): 
    archivo = parkinsons 
    tam_atributos = 22
    cantidad_atributos = 195
  if (int(i) == 3): 
    archivo = heart
    tam_atributos = 44
    cantidad_atributos = 349
  
  data, metadata = arff.loadarff(archivo)
  
  print()
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  print("Visualización de los datos excogidos")  
  print(metadata)
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  print()
  
  # Conversión de data a una matriz numpy y a un array paralelo de características
  datos = np.empty((cantidad_atributos, tam_atributos))
  
  for i in range(tam_atributos): 
    for j in range(cantidad_atributos): 
      datos[j][i] = data[j][i]
  
  labels = []
  for i in range(cantidad_atributos):
    labels.append(str(data[i][tam_atributos])[2])  
  
  # Ejecuciones con los diferentes algoritmos 
  NUMERO_ITERACIONES = 1500 
  print()
  print("Cálculo de AGG con BLX")
  print("...")
  print("Calculándo")
  print("...")
  pesos = agg(data, labels, blx, NUMERO_ITERACIONES)
  print("Pesos: {}".format(pesos))
if i == 0: 
  print()
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  print()
  print("Tablas parciales")
  print()
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
  print()
  print("AGG-BLX")
  print()
  print("Ionosphere")
  head = ["","%_clas", "%_red", "Agr", "T(ms)"]
  mydata = [
            ["Particion1", "", "", "", ""],
            ["Particion2", "", "", "", ""],
            ["Particion3", "", "", "", ""],
              ["Particion4", "", "", "", ""],
            ["Particion5", "", "", "", ""],
            ["Media", "", "", "", ""],
  ] 
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Parkinsons")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Spect-heart")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print() 
  print("AGG-CA")
  print()
  print("Ionosphere")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Parkinsons")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Spect-heart")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print()  
  print("AGE-BLX")
  print()
  print("Ionosphere")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Parkinsons")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Spect-heart")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print() 
  print("AGE-CA")
  print()
  print("Ionosphere")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Parkinsons")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print("Spect-heart")
  print(tabulate(mydata, headers=head, tablefmt="grid"))
  print()   

¿Qué datos quiere leer?
1- ionosphere
2- parkinsons
3- spectf-heart
0- generacion de datos para tablas
Introduzca 0, 1, 2, 3: 1

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Visualización de los datos excogidos
Dataset: ionosphere
	a01's type is numeric
	a02's type is numeric
	a03's type is numeric
	a04's type is numeric
	a05's type is numeric
	a06's type is numeric
	a07's type is numeric
	a08's type is numeric
	a09's type is numeric
	a10's type is numeric
	a11's type is numeric
	a12's type is numeric
	a13's type is numeric
	a14's type is numeric
	a15's type is numeric
	a16's type is numeric
	a17's type is numeric
	a18's type is numeric
	a19's type is numeric
	a20's type is numeric
	a21's type is numeric
	a22's type is numeric
	a23's type is numeric
	a24's type is numeric
	a25's type is numeric
	a26's type is numeric
	a27's type is numeric
	a28's type is numeric
	a29's type is numeric
	a30's type is numeric
	a31's type is numeric
	a32's type is numeric
	a33's type is numeric
	a34's type is num